## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [16]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.tail()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
169,469,Saint-Joseph,RE,78.80,light rain,-21.3667,55.6167,"""Plantation Bed and Breakfast"
170,472,Say,NE,87.80,clear sky,13.1007,2.3689,NaN
171,473,Port Hedland,AU,84.20,broken clouds,-20.3167,118.5667,The Esplanade Hotel
172,475,Bubaque,GW,77.83,broken clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
173,480,Celestun,MX,78.89,overcast clouds,20.8667,-90.4000,Posada Lilia


In [18]:
preferred_vacation_df = vacation_df.dropna()
preferred_vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
1,3,Las Vegas,US,87.80,clear sky,36.1750,-115.1372,Plaza Hotel & Casino
2,10,Mahebourg,MU,77.00,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
3,14,San Lorenzo,PY,78.01,clear sky,-25.3333,-57.5333,Hotel del Sur
4,16,Avarua,CK,84.20,broken clouds,-21.2078,-159.7750,Paradise Inn


In [19]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
preferred_vacation_info = [info_box_template.format(**row) for index, row in preferred_vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = preferred_vacation_df[["Lat", "Lng"]]

In [20]:
# 4a. Add a marker layer for each city to the map.
locations = preferred_vacation_df[["Lat", "Lng"]]
max_temp = preferred_vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=preferred_vacation_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [22]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

preferred_vacation_start = preferred_vacation_df.loc[preferred_vacation_df['City']=='Acapulco']
preferred_vacation_end = preferred_vacation_df.loc[preferred_vacation_df['City']=='Acapulco']
preferred_vacation_stop1 = preferred_vacation_df.loc[preferred_vacation_df['City']=='Ixtapa']
preferred_vacation_stop2 = preferred_vacation_df.loc[preferred_vacation_df['City']=='Puerto Escondido'] 
preferred_vacation_stop3 = preferred_vacation_df.loc[preferred_vacation_df['City']=='Cabo San Lucas'] 

In [24]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = preferred_vacation_start["Lat"].to_numpy()[0],preferred_vacation_start["Lng"].to_numpy()[0]
end = preferred_vacation_start["Lat"].to_numpy()[0],preferred_vacation_start["Lng"].to_numpy()[0]
stop1 = preferred_vacation_stop1["Lat"].to_numpy()[0], preferred_vacation_stop1["Lng"].to_numpy()[0]
stop2 = preferred_vacation_stop2["Lat"].to_numpy()[0], preferred_vacation_stop2["Lng"].to_numpy()[0]
stop3 = preferred_vacation_stop3["Lat"].to_numpy()[0], preferred_vacation_stop3["Lng"].to_numpy()[0]
print(f'Start: {start}, End: {end}')

print(f'Stop 1: {stop1}, Stop 2: {stop2}, Stop 3: {stop3}')

Start: (16.8634, -99.8901), End: (16.8634, -99.8901)
Stop 1: (20.7, -105.2), Stop 2: (15.85, -97.0667), Stop 3: (22.8909, -109.9124)


In [25]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
preferred_vacation_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')

fig.add_layer(preferred_vacation_itinerary)
fig


Figure(layout=FigureLayout(height='420px'))

In [26]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([preferred_vacation_start,preferred_vacation_stop1,preferred_vacation_stop2,preferred_vacation_stop3 ],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,327,Acapulco,MX,82.4,scattered clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
1,285,Ixtapa,MX,77.0,broken clouds,20.7000,-105.2000,Hotel Plaza Providencia
2,390,Puerto Escondido,MX,84.2,clear sky,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
3,18,Cabo San Lucas,MX,82.4,scattered clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos


In [27]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(10.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))